In [ ]:
"""
Load a BehaviourStore setup json and interactively edit it.
"""
import os
from behaviourstoreeditor.behaviourentryeditor import *

In [ ]:
VBox([BehaviourEntryEditor() for i in range(3)])

In [ ]:
file_ext = ".behaviourstore.json"

output_field = Output()

behaviour_store_folder_field = Text(    
    value=os.getcwd(),
    placeholder=os.getcwd(),
    description='Folder:',
    layout=Layout(width='100%')
)

loadbutton = Button(
    tooltip='Load store files',
    disabled=False,
    icon='upload',
    layout=Layout(width='100%')
)

fileselector = Select(
    options=[],
    description='Read file:',
    disabled=True
)

reloadbutton = Button(
    tooltip='Reload from file',
    disabled=False,
    icon=icon_reload,
    layout=Layout(width='100%')
)

deletebutton = Button(
    tooltip='Delete store file',
    disabled=False,
    icon=icon_delete,
    layout=Layout(width='100%')
)

filename_for_saving_field = Text(    
    value="new_file_name",
    placeholder="file_name",
    description='Write file:',
    layout=Layout(width='100%')
)

createbutton = Button(
    tooltip='Create store file',
    disabled=False,
    icon=icon_create,
    layout=Layout(width='100%')
)

savebutton = Button(
    tooltip='Save to file',
    disabled=False,
    icon=icon_save,
    layout=Layout(width='100%')
)


def reload_file_selector(button):
    # load file selector and update current working folder
    try:
        fileselector.options = [fil[:-len(".behaviourstore.json")] for fil in os.listdir(behaviour_store_folder_field.value) if
                                fil.endswith(file_ext)]
    except:
        fileselector.disabled=True
        with output_field:
            print("somethng bad happened")
    finally:
        fileselector.disabled=False

loadbutton.on_click(reload_file_selector)


def file_path_invalid():
    # check if behaviour_store_folder_field.value is ok
    return False
        

def create_button_click(button):
    if file_path_invalid():
        with output_field:
            print("path invalid!")
        return
    with open(behaviour_store_folder_field.value + "/" +
                 filename_for_saving_field.value +
                  file_ext, 'w'):
        pass
    reload_file_selector(None)
    fileselector.value=filename_for_saving_field.value
    
createbutton.on_click(save_button_click)
    
def delete_button_click(button):
    if file_path_invalid():
        with output_field:
            print("file path invalid!")
        return
    try:
        os.remove(behaviour_store_folder_field.value + "/" +
                 fileselector.value +
                  file_ext
                 )
    except:
        pass
    reload_file_selector(None)
    
deletebutton.on_click(delete_button_click)


VBox(children=[
    MakeHBox_single([behaviour_store_folder_field, loadbutton], ['95%','5%']),
    MakeHBox_single([fileselector, reloadbutton, deletebutton], 
         ['90%', '5%', '5%']),
    MakeHBox_single([filename_for_saving_field, savebutton, createbutton],
                   ['90%', '5%', '5%']),
    output_field
    ]
)


